In [1]:
# Import Dependencies
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
import time

In [56]:
# Set the executable path and initialize Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Get LATEST driver version for 93.0.4577
Trying to download new driver from https://chromedriver.storage.googleapis.com/93.0.4577.63/chromedriver_mac64.zip
Driver has been saved in cache [/Users/curtissmith/.wdm/drivers/chromedriver/mac64/93.0.4577.63]


In [57]:
# Visit the Beer Advocate Top 250 page
url = 'https://www.beeradvocate.com/beer/styles/'
browser.visit(url)

In [58]:
# Convert the browser html to a soup object and then quit the browser
html = browser.html
styles_home = soup(html, 'html.parser')

In [59]:
# Find all anchor tags
all_hrefs = styles_home.find_all('a')

In [60]:
styles_hrefs = []
error_hrefs = ['<a href="/beer/styles/">Beer Styles</a>',
              '<a href="/beer/styles/#XenForo"><i class="uix_icon uix_icon-jumpToTop"></i> <span class="uix_hide">Top</span></a>',
              '<a href="/beer/top-styles/">Beers by Style</a>',
              '<a href="/beer/top-styles/">Styles</a>']
for item in all_hrefs:
    if "/beer/styles/" in str(item):
        if str(item) not in error_hrefs:
            styles_hrefs.append(item.get('href'))

In [63]:
start_time = time.time()

count = 0
num_skipped = 0
errors = []
exception = []
beer_info = []
mininum_num_reviews = 75


browser.visit(url)
styles_home = soup(html, 'html.parser')


#iterate through style links
for style_link in styles_hrefs:
    print(f'Style Link: {style_link}')
    print(f'Number of beers: {len(beer_info)}')
    length = (time.time() - start_time)
    print(f'Elapsed Time: {length} seconds / {length/60} minutes')
    
    try:
    
        # Reseting num_reviews_over
        under_min_num = 0
        
        next_page_links = [style_link,
                f"{style_link}?sort=revsD&start=50",]
        
        for page in next_page_links:
            
            # Setting url variable to navigate back to
            style_url = f"https://www.beeradvocate.com{page}"
            
            # If the number of reviews is under the limit, go to next style
            if num_reviews_under == True:
                browser.visit(url)
                styles_home = soup(html, 'html.parser')
                continue
                
            if under_min_num == 10:
                    
                browser.visit(url)
                styles_home = soup(html, 'html.parser')
                
                break
        
            # navigate to link
            browser.click_link_by_href(page)

            # Parse the beer html with soup
            html = browser.html
            style_profile = soup(html, 'html.parser')

            # find all a tags
            all_a = style_profile.find_all('a')

            # select only relevant link (those with "beer/profile" in link)
            links = [x for x in all_a if '/beer/profile/' in str(x)]

            # iterate and get links to beers - even indices are links to beers, odd are links to brewery
            beer_a_tags = []
            for index, item in enumerate(links):
                if index % 2 == 0:
                    beer_a_tags.append(item)

            # get href url for navigating to beer webpage
            beer_href = [x.get('href') for x in beer_a_tags]

            #iterate through links
            for link in beer_href:

                # navigate to link
                browser.click_link_by_href(link)

                # Parse the beer html with soup
                html = browser.html
                beer_soup = soup(html, 'html.parser')
#                 print(browser.url)
#                 print(beer_soup.find_all('a'))
                
                try:

                    reviews = []

                    spans = beer_soup.find_all('span', class_='Tooltip')
                    for count, item in enumerate(spans):
                        if "Reviews:" in str(item):
                            num_reviews = spans[count+1].text
                            num_reviews = int(num_reviews.replace(",", ""))

                    # Only have beers with > 100 reviews
                    if num_reviews < mininum_num_reviews:
    #                     print(num_reviews)
                        num_skipped += 1
                        under_min_num += 1
            
                        if under_min_num == 10:
                    
                            browser.visit(url)
                            styles_home = soup(html, 'html.parser')
                            break

                        browser.visit(style_url)

                        # Parse the beer html with soup
                        html = browser.html
                        style_profile = soup(html, 'html.parser')

                        continue

#                     sorting_links = [f"{link}?sort=top",
#                                     f"{link}?view=beer&sort=top&start=25",
#                                     f"{link}?view=beer&sort=top&start=50",
#                                     f"{link}?view=beer&sort=top&start=75"]

                    sorting_links = [f"{link}?sort=top",
                                    f"{link}?view=beer&sort=top&start=25",
                                    f"{link}?view=beer&sort=top&start=50",
                                    ]


                    for item in sorting_links:

#                         print(browser.url)
                        
                        browser.click_link_by_href(item)

                        html = browser.html
                        beer_soup = soup(html, 'html.parser')

                        # Get beer and brewery
                        header = beer_soup.find('div', class_='titleBar')
                        brewery = header.find("span").text
                        beer_name = header.text.replace(brewery, "")
                        beer_name = beer_name.replace('\n', "")

                        # get ABV and rating
                        spans = beer_soup.find_all('span', class_='Tooltip')
#                         print(spans)
                        abv = spans[3].text
                        rating = spans[7].text

                        # get style
                        a = beer_soup.find_all('a', class_='Tooltip')
                        style = a[4].text

                        # get reviews
                        review_div = beer_soup.find_all('div', {"id": "rating_fullview_content_2"})
                        for item in review_div:
                            reviews.append(item.text.split("%")[1])

                    # create dictionary and add to list
                    beer_info.append({"beer_name": beer_name,
                                      "beer_style": style,
                                      "brewery": brewery,
                                      "abv": abv,
                                      "rating": rating,
                                      "reviews": reviews})

                    #print(len(reviews))

                    browser.visit(style_url)

                    # Parse the beer html with soup
                    html = browser.html
                    style_profile = soup(html, 'html.parser')                       
            
                except Exception as exception:
                    print(exception)
                    exceptions.append(exception)
                    errors.append(beer_name)
                    print(f'Current Error Count: {len(errors)}')


                    if browser.url == 'https://www.google.com/':
                        break

                    browser.visit(style_url)

                    # Parse the beer html with soup
                    html = browser.html
                    style_profile = soup(html, 'html.parser') 
                    
    except Exception as e:
        print(e)

    if browser.url == 'https://www.google.com/':
        break

    browser.visit(url)

    # Parse the beer html with soup
    browser.visit(url)
    styles_home = soup(html, 'html.parser')
    
length = (time.time() - start_time)
print(f'Script ran in {length} seconds / {length/60} minutes')
print(f'Obtained {len(beer_info)} beer records')
print(f'Skipped {num_skipped} beers')
print(f'There were {len(errors)} errors')

Style Link: /beer/styles/35/
Number of beers: 0
Elapsed Time: 1.3114500045776367 seconds / 0.021857500076293945 minutes
Style Link: /beer/styles/36/
Number of beers: 43
Elapsed Time: 262.83366894721985 seconds / 4.3805611491203305 minutes
Style Link: /beer/styles/33/
Number of beers: 47
Elapsed Time: 300.7869801521301 seconds / 5.013116335868835 minutes
Style Link: /beer/styles/32/
Number of beers: 66
Elapsed Time: 421.53896713256836 seconds / 7.025649452209473 minutes
Style Link: /beer/styles/92/
Number of beers: 85
Elapsed Time: 544.0147478580475 seconds / 9.066912464300792 minutes
Style Link: /beer/styles/86/
Number of beers: 99
Elapsed Time: 631.647302865982 seconds / 10.527455047766368 minutes
Style Link: /beer/styles/73/
Number of beers: 117
Elapsed Time: 736.8599200248718 seconds / 12.280998667081198 minutes
Style Link: /beer/styles/119/
Number of beers: 166
Elapsed Time: 991.7314651012421 seconds / 16.52885775168737 minutes
Style Link: /beer/styles/74/
Number of beers: 177
Elap

Style Link: /beer/styles/70/
Number of beers: 2120
Elapsed Time: 12183.570469856262 seconds / 203.0595078309377 minutes
Style Link: /beer/styles/168/
Number of beers: 2128
Elapsed Time: 12239.921753168106 seconds / 203.9986958861351 minutes
Style Link: /beer/styles/8/
Number of beers: 2128
Elapsed Time: 12256.785785198212 seconds / 204.27976308663685 minutes
Style Link: /beer/styles/173/
Number of beers: 2150
Elapsed Time: 12378.273903131485 seconds / 206.3045650521914 minutes
Style Link: /beer/styles/169/
Number of beers: 2151
Elapsed Time: 12400.822005033493 seconds / 206.68036675055822 minutes
Style Link: /beer/styles/5/
Number of beers: 2158
Elapsed Time: 12450.836230039597 seconds / 207.51393716732662 minutes
Style Link: /beer/styles/72/
Number of beers: 2164
Elapsed Time: 12497.373471975327 seconds / 208.28955786625545 minutes
Style Link: /beer/styles/12/
Number of beers: 2226
Elapsed Time: 12795.679271936417 seconds / 213.26132119894027 minutes
Style Link: /beer/styles/11/
Numbe

In [65]:
print(errors)

[]


In [66]:
df = pd.DataFrame(beer_info)

In [67]:
df

,beer_name,beer_style,brewery,abv,rating,reviews
0,Ayinger Celebrator,Bock - Doppelbock,Ayinger Privatbrauerei,6.7%,4.37,[look: 4 | smell: 4 | taste: 4 | feel: 4.25 | ...
1,Troegenator,Bock - Doppelbock,Tröegs Brewing Company,8.2%,3.97,[look: 4 | smell: 4 | taste: 4 | feel: 4 | ove...
2,Spaten Optimator,Bock - Doppelbock,Spaten-Franziskaner-Bräu,7.6%,3.92,[look: 3.75 | smell: 3.75 | taste: 3.75 | feel...
3,Salvator,Bock - Doppelbock,Paulaner Brauerei,7.9%,3.96,[look: 3.5 | smell: 3.5 | taste: 4 | feel: 4 |...
4,Korbinian,Bock - Doppelbock,Bayerische Staatsbrauerei Weihenstephan,7.4%,4.21,[look: 4 | smell: 4 | taste: 4 | feel: 4 | ove...
...,...,...,...,...,...,...
3515,Cuvée De Castleton,Wild Ale,Captain Lawrence Brewing Co.,8.4%,4.39,[look: 3.5 | smell: 3.5 | taste: 3.75 | feel: ...
3516,The Wild One,Wild Ale,Bell's Brewery - Eccentric Café & General Store,6.3%,4.17,[look: 4 | smell: 4.25 | taste: 4 | feel: 4 | ...
3517,Figaro,Wild Ale,Cascade Brewing / Raccoon Lodge & Brewpub,9.5%,4.21,"[maybe the best ive had from cascade, which is..."
3518,Raspberry,Wild Ale,Upland Brewing Company,6%,4.18,[look: 4 | smell: 4.5 | taste: 4.25 | feel: 4....


In [68]:
df.to_csv("new_scraped_data_v2.csv", index=False)